<h2> Function Calling: Zazzle API meets ChatGPT</h2>

<h4> Importing Relevant Files</h4>

In [221]:
# Importing the relevant Packages 
from openai import AzureOpenAI
import os
import requests
import json
import urllib.parse

<h4>Creating the Messages to help AI responses</h4>

In [232]:
# Take the User Input
event_name = input("What is the name/content of the event that you want merch for: ")
print(event_name)

What is the name/content of the event that you want merch for:  Tao's 30th Birthday


Tao's 30th Birthday


<h4> Setting up the Credentials </h4>

In [233]:
client = AzureOpenAI(
    api_key = os.getenv("AZURE_KEY"),
    azure_endpoint = os.getenv("AZURE_ENDPOINT"),
    api_version = "2023-10-01-preview"
)

<h4> Create  function to Generate a Poem for this event </h4>

In [234]:
def generate_poem_from_chatgpt(event_name):
    messages = [
        {"role": "system", "content": f"Generate a 130 character rap for {event_name}."},
        {"role": "user", "content": f"Write a rap for {event_name}"}
    ]
    response = client.chat.completions.create(
        model="GPT-4",
        messages=messages,
        tools=functions,  # Note this
        tool_choice="auto"  # And this
    )
    poem = response.choices[0].message.content # Access the generated poem (from the 'message' field in the first choice)
    return poem

<h4> Create a Function to create a Zazzle Link for a Mug that has the poem printed on it </h4>

In [235]:
def mug_creator(event_name):
    poem = generate_poem_from_chatgpt(event_name)
    # Base URL for the Coffee Mug creation on Zazzle
    base_url = "https://www.zazzle.com/api/create/at-238348553720143566?rf=238348553720143566&pd=256529855831094801&ed=true&ax=Linkover&t_customblanktemplate_txt="
    encoded_poem = urllib.parse.quote_plus(poem)     # URL encode the entire poem (preserving spaces as '%20')
    final_url = base_url + encoded_poem
    return final_url    # Return the final Zazzle URL

In [236]:
functions = [
    {
        "type": "function",
        "function": {
            "name": "get_zazzle_link",
            "description": "Get a zazzle link for the event",
            "parameters": {
                "type": "object",
                "properties": {
                    "zazzle_link": {
                        "type": "string",
                        "description": "The name of the event for which a mug will be designed."
                    }
                },
                "required": ["zazzle_link"]
            }
        }
    }
]

In [237]:
messages = [
    {"role": "system", "content": f"Generate a personalized Zazzle Link for {event_name}"},
    {"role": "user", "content": f"Give me a personalized zazzle link for my event: {event_name}"}
]

In [238]:
response = client.chat.completions.create(
    model="GPT-4",
    messages=messages,
    tools=functions,
    tool_choice="auto"
)

In [239]:
response_message = response.choices[0].message
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vzYN29KA23P7jEf2kwbNR2Lm', function=Function(arguments='{\n  "zazzle_link": "Tao\'s 30th Birthday"\n}', name='get_zazzle_link'), type='function')])


In [240]:
gpt_tools = response.choices[0].message.tool_calls
print(gpt_tools)

[ChatCompletionMessageToolCall(id='call_vzYN29KA23P7jEf2kwbNR2Lm', function=Function(arguments='{\n  "zazzle_link": "Tao\'s 30th Birthday"\n}', name='get_zazzle_link'), type='function')]


In [241]:
if gpt_tools:
    available_functions = {
        "get_zazzle_link": mug_creator
    }
    messages.append(response_message)
    for gpt_tool in gpt_tools:
        function_name = gpt_tool.function.name
        print(function_to_call)
        function_parameters = json.loads(gpt_tool.function.arguments)
        function_response = function_to_call(function_parameters.get('zazzle_link')) # applies the get_zazzle_link parameters to the mug_creator function
        print(function_parameters.get('zazzle_link'))
        messages.append(
            {
                "tool_call_id": gpt_tool.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            }
        )
        second_response = client.chat.completions.create(
            model="GPT-4",
            messages=messages
        )
        print(second_response.choices[0].message.content)
else:
    print(response.choices[0].message.content)

<function mug_creator at 0x00000217F517EAC0>
Tao's 30th Birthday
Here's your personalized Zazzle link for Tao's 30th Birthday: [Click here](https://www.zazzle.com/api/create/at-238348553720143566?rf=238348553720143566&pd=256529855831094801&ed=true&ax=Linkover&t_customblanktemplate_txt=%22Hey%2C+it%27s+Tao%27s+day%2C+big+30+on+display%2C+wisdom+in+his+DNA.+Aging+like+fine+Cabernet%2C+older+but+heart%27s+still+play.+HBD%2C+Tao%2C+let%27s+slay%21%22)
